## Task

1. Define a $2\times2$ or $3\times3$ matrix $A$.
2. Find the characteristic polynomial and minimal polynomial.
3. Use the Cayley-Hamilton theorem to express $e^{At}$ as a polynomial in $A$.
4. Compare the result with `scipy.linalg.expm`.